<a href="https://colab.research.google.com/github/xAngel563x/ABP/blob/master/Tutorial_an%C3%A1lisis_de_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplo de aplicación de un algoritmo de clasificación supervisado usando un dataset para el análisis de sentimientos



In [ ]:
import pandas as pd

trainingData = pd.read_csv('semeval-2017-train.csv', delimiter='	')
trainingData = trainingData.head(1000) #Eliminar la funcion head() si se quiere usar todo el dataset. Para las pruebas usamos únicamente los 1000 primeros tweets
trainingData

FileNotFoundError: [Errno 2] No such file or directory: 'semeval-2017-train.csv'

In [ ]:
trainingData['label'].value_counts()

### Preprocesamiento de los datos de entrenamiento

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

for row in trainingData.itertuples():


    text = word_tokenize(row[2]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedText.append(text)

preprocessedData = trainingData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

### Creación de la bolsa de palabras

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

In [ ]:
textsBoW.shape

###Entrenamiento de un algoritmo de clasificación (SVM)



In [ ]:
from sklearn import svm
svc = svm.SVC(kernel='linear') #Modelo de clasificación

X_train = textsBoW #Documentos
Y_train = trainingData['label'] #Etiquetas de los documentos
svc.fit(X_train, Y_train) #Entrenamiento

### Carga de y preprocesado de documentos de test

In [ ]:
testData = pd.read_csv('semeval-2017-test.csv', delimiter='	')
testData = testData.head(100)
testData

In [ ]:
ps = PorterStemmer()

preprocessedText = []

for row in testData.itertuples():


    text = word_tokenize(row[2]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedText.append(text)

preprocessedDataTest = testData
preprocessedDataTest['processed_text'] = preprocessedText

preprocessedDataTest

In [ ]:
testData['label'].value_counts()

Importante!:
Se emplea la misma representación de bolsa de palabras que se ha usado para entrenar (bagOfWordsModel)

In [ ]:
textsBoWTest= bagOfWordsModel.transform(preprocessedDataTest['processed_text'])
print("Finished")

In [ ]:
textsBoWTest.shape

### Clasificación de los documentos de test

In [ ]:
X_test = textsBoWTest #Documentos


predictions = svc.predict(X_test) #Se almacena en el array predictions las predicciones del clasificador

###Evaluación de la predicción

In [ ]:
from sklearn.metrics import classification_report

Y_test = testData['label'] #Etiquetas reales de los documentos

print (classification_report(Y_test, predictions))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)


neigh.fit(X_train, Y_train)
predictions = neigh.predict(X_test)

print (classification_report(Y_test, predictions))